<a href="https://colab.research.google.com/github/Samoed/CourseWork2/blob/main/Test_lenngth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U bert-extractive-summarizer
!pip3 install -U transformers

     |████████████████████████████████| 2.0MB 5.4MB/s 
     |████████████████████████████████| 890kB 28.4MB/s 
     |████████████████████████████████| 3.2MB 19.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=aee48658c5bc81dd5dcd86b3a3ae493e7ac3f7badf2d43ff38984ea738558114
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)


In [ ]:
!unzip embeddingsShort.zip

In [5]:
import requests
import torch
from transformers import BertTokenizer, BertModel, BertConfig
from tqdm import tqdm
from summarizer import Summarizer

#from PyQt5 import QtCore, QtGui, QtWidgets
#from PyQt5.QtWidgets import QMessageBox
import sys

import pickle
import re
import math
import os

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#print(torch.cuda.get_device_name(0))

custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
#custom_model.to('cuda')
custom_model.eval()

modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [11]:
path = '/content/embeddingsPickle'
directories = os.listdir(path)
emb = {}
for file in directories:
    with open(path+'/'+file, "rb") as f:
        arr = pickle.load(f)
        emb[file.split('.')[0]] = arr

In [13]:
with open("TestDoc", 'r') as f:
    text = f.read()
    text = "".join([i.replace('\n',' ').replace('\t',' ') for i in text])
text
%time embeddings = modelSum.run_embeddings(text, num_sentences=10)

CPU times: user 1min 1s, sys: 1.49 s, total: 1min 2s
Wall time: 1min 2s


# Сумма всех

In [32]:
errorSum = {}
errorMae = {}
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    sum = 0
    mae = 0
    mse = 0
    for i in range(len(embeddings)):
        for arrRow in range(len(arr)):
            for j in range(768):
                sum += embeddings[i][j] - arr[arrRow][j]
                mae += abs(embeddings[i][j] - arr[arrRow][j])
                mse += (embeddings[i][j] - arr[arrRow][j])**2
    errorSum[sum] = name
    errorMae[mae] = name
    errorMse[mse] = name
print()
print(errorSum[min(errorSum.keys())], errorMae[min(errorMae.keys())], errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:39<00:00,  1.55it/s]


 Кычкин Алексей Владимирович  Кушев Вадим Олегович  Кушев Вадим Олегович


# Сумма минимальных предлжений с повторами

In [40]:
errorSum = {}
errorMae = {}
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allSum = 0
    allMae = 0
    allMse = 0
    for i in range(len(embeddings)):
        sum = [0]*len(arr)
        mae = [0]*len(arr)
        mse = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                sum[arrRow] += embeddings[i][j] - arr[arrRow][j]
                mae[arrRow] += abs(embeddings[i][j] - arr[arrRow][j])
                mse[arrRow] += (embeddings[i][j] - arr[arrRow][j])**2
        allSum += min(sum)
        allMae += min(mae)
        allMse += min(mse)
    errorSum[allSum] = name
    errorMae[allMae] = name
    errorMse[allMse] = name
print()
print(errorSum[min(errorSum.keys())], errorMae[min(errorMae.keys())], errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:40<00:00,  1.52it/s]


 Божья-Воля Анастасия Александровна  Кычкин Алексей Владимирович  Кычкин Алексей Владимирович


# Сумма минимальных предложений без повторов

## Avg

In [44]:
errorSum = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allSum = 0
    takenRow = []
    for i in range(len(embeddings)):
        if i in takenRow:
            continue
        sum = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                sum[arrRow] += embeddings[i][j] - arr[arrRow][j]
        allSum += min(sum)
        takenRow.append(sum.index(min(sum)))
    errorSum[allSum] = name
print()
print(errorSum[min(errorSum.keys())])

100%|██████████| 61/61 [00:06<00:00,  9.01it/s]


 Божья-Воля Анастасия Александровна


## Mae

In [46]:
errorMae = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allMae = 0
    takenRow = []
    for i in range(len(embeddings)):
        if i in takenRow:
            continue
        mae = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                mae[arrRow] += embeddings[i][j] - arr[arrRow][j]
        allMae += min(mae)
        takenRow.append(mae.index(min(mae)))
    errorMae[allMae] = name
print()
print(errorMae[min(errorMae.keys())])

100%|██████████| 61/61 [00:06<00:00,  8.90it/s]


 Божья-Воля Анастасия Александровна


## Mse

In [47]:
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allMse = 0
    takenRow = []
    for i in range(len(embeddings)):
        if i in takenRow:
            continue
        mse = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                mse[arrRow] += embeddings[i][j] - arr[arrRow][j]
        allMse += min(mse)
        takenRow.append(mse.index(min(mse)))
    errorMse[allMse] = name
print()
print(errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:06<00:00,  8.86it/s]


 Божья-Воля Анастасия Александровна


# Усредение эмбединнгов

In [54]:
newEmb = [0]*768
for arrEmb in embeddings:
    for j in range(len(arrEmb)):
        newEmb[j] += arrEmb[j] 
newEmb = [i/10 for i in newEmb]
errorSum = {}
errorMae = {}
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    if name==" Лядова Людмила Николаевна":
        continue
    sum = 0
    mae = 0
    mse = 0
    newArr = [0]*768
    for arrEmb in arr:
        for j in range(len(arrEmb)):
            newArr[j] += arrEmb[j] 
    newArr = [i/10 for i in newArr]
    for j in range(768):
        sum += newEmb[j] - newArr[j]
        mae += abs(newEmb[j] - newArr[j])
        mse += (newEmb[j] - newArr[j])**2
    errorSum[sum] = name
    errorMae[mae] = name
    errorMse[mse] = name
print()
print(errorSum[min(errorSum.keys())], errorMae[min(errorMae.keys())], errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:00<00:00, 124.93it/s]


 Кычкин Алексей Владимирович  Кузнецов Денис Борисович  Кузнецов Денис Борисович
